# Combinatorics Puzzle

Write a function answer(w, h, s) that takes 3 integers and returns the number of unique, non-equivalent configurations that can be found on a grid **w** blocks wide and **h** blocks tall where each body has **s** possible states. Equivalency is defined as: any two grids with each body in the same state where the actual order of the rows and columns do not matter (and can thus be freely swapped around). Grid standardization means that the width and height of the grid will always be between 1 and 12, inclusive. And while there are a variety of bodies in each grid, the number of states of those bodies is between 2 and 20, inclusive. The answer can be over 20 digits long, so return it as a decimal string.  The intermediate values can also be large, so you will likely need to use at least 64-bit integers.

For example, consider w=2, h=2, s=2. We have a 2x2 grid where each body is either in state 0 (for instance, silent) or state 1 (for instance, noisy).  We can examine which grids are equivalent by swapping rows and columns.

00<br>
00

In the above configuration, all bodies are "silent" - that is, they have a state of 0 - so any swap of row or column would keep it in the same state.

00 00 01 10<br>
01 10 00 00

1 body is emitting noise - that is, has a state of 1 - so swapping rows and columns can put it in any of the 4 positions.  All four of the above configurations are equivalent.

00 11<br>
11 00

2 bodies are emitting noise side-by-side.  Swapping columns leaves them unchanged, and swapping rows simply moves them between the top and bottom.  In both, the *groupings* are the same: one row with two bodies in state 0, one row with two bodies in state 1, and two columns with one of each state.

01 10<br>
01 10

2 noisy bodies adjacent vertically. This is symmetric to the side-by-side case, but it is different because there's no way to transpose the grid.

01 10<br>
10 01

2 noisy bodies diagonally.  Both have 2 rows and 2 columns that have one of each state, so they are equivalent to each other.

01 10 11 11<br>
11 11 01 10

3 noisy bodies, similar to the case where only one of four is noisy.

11<br>
11

4 noisy bodies.

There are 7 distinct, non-equivalent grids in total, so answer(2, 2, 2) would return 7.


## Test cases

Inputs:<br>
    (int) w = 2<br>
    (int) h = 2<br>
    (int) s = 2<br>
    
Output:<br>
    (string) "7"<br>

Inputs:<br>
    (int) w = 2<br>
    (int) h = 3<br>
    (int) s = 4<br>

Output:<br>
    (string) "430"<br>

# Solution Basis

The two keys to this puzzle:
1. The representation of how to count the combinations - in this case the use of the cycle index of the symmetric group.
2. Using tuples for the mulitplaction of coefficients where the supersripts and subscripts were combined in different ways.
    a. the superscript used greatest common denominator (gcd)
    b. the subscript used lowest common multiple (lcm)

## The Cycle Index of a Symmetric Group $Z(S_n)$

In order to find the solution, each dimension of the grid is represented by the cycle index of a symetric group for that dimention. (e.g. a 2x3 grid is represnted by $Z(S_2)$ and $Z(S_3)$). The formula for the cycle index of a symmetric group is as follows:

$$Z(S_n) = \frac{1}{n} \sum_{l=1}^n a_l \; Z(S_{n-l})$$
from: https://en.wikipedia.org/wiki/Cycle_index


$Z(S_0)=1$
$Z(S_1)=\frac{1}{1}\; a_1 \; Z(S_0) = a_1$ 

$Z(S_2) = \frac{1}{2} \sum_{l=1}^2 a_l \; Z(S_{n-l}) = \frac{1}{2} \big( a_1\;Z(S_{2-1}) + a_2\;Z(S_{2-2})       \big) = \frac{1}{2} (a_1^2 + a_2)$

$Z(S_3) = \frac{1}{3} \sum_{l=1}^3 a_l \; Z(S_{n-l})$

$$Z(S_3)= \frac{1}{3} \big( a_1\;Z(S_{2}) + a_2\;Z(S_{1}) + a_3\;Z(S_0) \big) $$

$$ Z(S_3)= \frac{1}{3} \left( a_1\;\frac{1}{2} \big(a_1^2 + a_2 \big) + a_2a_1 + a_3\ \right) $$

$$ Z(S_3)= \frac{1}{3} \left(\frac{1}{2} \big(a_1^3 + a_1  a_2 \big) + a_2 a_1 + a_3\ \right) $$

$$ Z(S_3)= \frac{1}{6} \big( a_1^3 + 3 a_1 a_2 + 2 a_3 \big) $$

and so on...

$$ Z(S_4)= \frac{1}{24} \big( a_1^4 + 6 a_1^2 a_2 + 8 a_3 a_1 + 3a_2^2 + 6 a_4 \big) $$

$$ Z(S_5)= \frac{1}{120} \big( a_1^5 + 10 a_1^3 a_2 + 20 a_3 a_1^2 + 15a_2^2 a_1 + 30 a_4 a_1 + 20 a_2 a_3 + 24 a_5 \big) $$

$$ ... $$

### Put Cycle Indices in a dictionary - cycle_index
For the function created, the cycle indices for 1 through 12 based on the above formula where placed in a dictionary ***cycle_index***.

## Combining the Cycle Indices

The key to combining the cycle indices in terms of  $Z(G\  X\ H)$ is in the combining the product of each variable set.

$$ a_i^m a_j^q = a_{lcm(i,j)}^{mq \ gcd(i,j)} (1)$$

This was handled in the code by use of tuples for each coefficient: (num,den,sub,sup)

    * num, dem: the numerator and denominator in front of the cofficient
    * sub, sup: the subscript and superscript of each coefficient


For example, $Z(S_2) = \frac{1}{2} (a_1^2 + a_2)$ is represented by (1,2,1,2) and (1,2,2,1).

*** Reference: 
(1) Peter J. Cameron, Daniele A. Gewurz, Francesca Merola, Product action, January 30, 2004*** 


# Solution Code

In [1]:
from fractions import Fraction

def gcd(a,b):
    while b > 0:
        a, b = b, a % b
    return long(a)

def lcm(a, b):
    return long(a * b / gcd(a, b))

def answer(w,h,s):

# create a dictionary to place all the 
# notation for cycle index monomials (numerator,denominator,subscript,superscript,...
# sub,sup,sub,sup,...)
# each tuple is a term in the cycle index
# for example for 2: it is 1/2 a_1^2 + 1/2 a_2^1
    cycle_index = {
        1: [(1,1,1,1)], 
        2: [(1,2,1,2),(1,2,2,1)], 
        3: [(1,6,1,3),(1,2,1,1,2,1),(1,3,3,1)],
        4: [(1,24,1,4),(1,4,1,2,2,1),(1,3,3,1,1,1),(1,8,2,2),(1,4,4,1)],
        5: [(1,120,1,5),(1,12,1,3,2,1),(1,6,3,1,1,2),(1,8,2,2,1,1),(1,4,4,1,1,1),
           (1,6,2,1,3,1),(1,5,5,1)],
        6: [(1,6,6,1),(1,18,3,2),(1,8,2,1,4,1),(1,48,2,3),(1,5,1,1,5,1),
           (1,6,1,1,2,1,3,1),(1,8,1,2,4,1),(1,16,1,2,2,2),(1,18,1,3,3,1),
           (1,48,1,4,2,1),(1,720,1,6)],
        7: [(1,7,7,1), (1,12,3,1,4,1),(1,10,2,1,5,1),(1,24,2,2,3,1),(1,6,1,1,6,1),
           (1,18,1,1,3,2),(1,8,1,1,2,1,4,1),(1,48,1,1,2,3),(1,10,1,2,5,1),
           (1,12,1,2,2,1,3,1),(1,24,1,3,4,1),(1,48,1,3,2,2),(1,72,1,4,3,1),
           (1,240,1,5,2,1),(1,5040,1,7)],
        8: [(1,8,8,1),(1,32,4,2),(1,15,3,1,5,1),(1,12,2,1,6,1),(1,36,2,1,3,2),
           (1,32,2,2,4,1),(1,384,2,4),(1,7,1,1,7,1),(1,12,1,1,3,1,4,1),
           (1,10,1,1,2,1,5,1),(1,24,1,1,2,2,3,1),(1,12,1,2,6,1),(1,36,1,2,3,2),
           (1,16,1,2,2,1,4,1),(1,96,1,2,2,3),(1,30,1,3,5,1),(1,36,1,3,2,1,3,1),
           (1,96,1,4,4,1),(1,192,1,4,2,2),(1,360,1,5,3,1),(1,1440,1,6,2,1),
           (1,40320,1,8)],
        9: [(1,9,9,1),(1,20,4,1,5,1),(1,18,3,1,6,1),(1,162,3,3),(1,14,2,1,7,1),
           (1,24,2,1,3,1,4,1),(1,40,2,2,5,1),(1,144,2,3,3,1),(1,8,1,1,8,1),
           (1,32,1,1,4,2),(1,15,1,1,3,1,5,1),(1,12,1,1,2,1,6,1),(1,36,1,1,2,1,3,2),
           (1,32,1,1,2,2,4,1),(1,384,1,1,2,4),(1,14,1,2,7,1),(1,24,1,2,3,1,4,1),
           (1,20,1,2,2,1,5,1),(1,48,1,2,2,2,3,1),(1,36,1,3,6,1),(1,108,1,3,3,2),
           (1,48,1,3,2,1,4,1),(1,288,1,3,2,3),(1,120,1,4,5,1),(1,144,1,4,2,1,3,1),
           (1,480,1,5,4,1),(1,960,1,5,2,2),(1,2160,1,6,3,1),(1,10080,1,7,2,1),
           (1,362880,1,9)],
        10:[(1,10,10,1),(1,50,5,2),(1,24,4,1,6,1),(1,21,3,1,7,1),(1,72,3,2,4,1),(1,16,2,1,8,1),
            (1,64,2,1,4,2),(1,30,2,1,3,1,5,1),(1,48,2,2,6,1),(1,144,2,2,3,2),(1,192,2,3,4,1),
            (1,3840,2,5),(1,9,1,1,9,1),(1,20,1,1,4,1,5,1),(1,18,1,1,3,1,6,1),(1,162,1,1,3,3),
            (1,14,1,1,2,1,7,1),(1,24,1,1,2,1,3,1,4,1),(1,40,1,1,2,2,5,1),(1,144,1,1,2,3,3,1),
            (1,16,1,2,8,1),(1,64,1,2,4,2),(1,30,1,2,3,1,5,1),(1,24,1,2,2,1,6,1),
            (1,72,1,2,2,1,3,2),(1,64,1,2,2,2,4,1),(1,768,1,2,2,4),(1,42,1,3,7,1),
            (1,72,1,3,3,1,4,1),(1,60,1,3,2,1,5,1),(1,144,1,3,2,2,3,1),(1,144,1,4,6,1),
            (1,432,1,4,3,2),(1,192,1,4,2,1,4,1),(1,1152,1,4,2,3),(1,600,1,5,5,1),
            (1,720,1,5,2,1,3,1),(1,2880,1,6,4,1),(1,5760,1,6,2,2),(1,15120,1,7,3,1),
            (1,80640,1,8,2,1),(1,3628800,1,10)],
        11:[(1,11,11,1),(1,30,5,1,6,1),(1,28,4,1,7,1),(1,24,3,1,8,1),(1,96,3,1,4,2),
            (1,90,3,2,5,1),(1,18,2,1,9,1),(1,40,2,1,4,1,5,1),(1,36,2,1,3,1,6,1),(1,324,2,1,3,3),
            (1,56,2,2,7,1),(1,96,2,2,3,1,4,1),(1,240,2,3,5,1),(1,1152,2,4,3,1),(1,10,1,1,10,1),
            (1,50,1,1,5,2),(1,24,1,1,4,1,6,1),(1,21,1,1,3,1,7,1),(1,72,1,1,3,2,4,1),
            (1,16,1,1,2,1,8,1),(1,64,1,1,2,1,4,2),(1,30,1,1,2,1,3,1,5,1),(1,48,1,1,2,2,6,1),
            (1,144,1,1,2,2,3,2),(1,192,1,1,2,3,4,1),(1,3840,1,1,2,5),(1,18,1,2,9,1),
            (1,40,1,2,4,1,5,1),(1,36,1,2,3,1,6,1),(1,324,1,2,3,3),(1,28,1,2,2,1,7,1),
            (1,48,1,2,2,1,3,1,4,1),(1,80,1,2,2,2,5,1),(1,288,1,2,2,3,3,1),(1,48,1,3,8,1),
            (1,192,1,3,4,2),(1,90,1,3,3,1,5,1),(1,72,1,3,2,1,6,1),(1,216,1,3,2,1,3,2),
            (1,192,1,3,2,2,4,1),(1,2304,1,3,2,4),(1,168,1,4,7,1),(1,288,1,4,3,1,4,1),
            (1,240,1,4,2,1,5,1),(1,576,1,4,2,2,3,1),(1,720,1,5,6,1),(1,2160,1,5,3,2),
            (1,960,1,5,2,1,4,1),(1,5760,1,5,2,3),(1,3600,1,6,5,1),(1,4320,1,6,2,1,3,1),
            (1,20160,1,7,4,1),(1,40320,1,7,2,2),(1,120960,1,8,3,1),(1,725760,1,9,2,1),
            (1,39916800,1,11)],
        12:[(1,12,12,1),(1,72,6,2),(1,35,5,1,7,1),(1,32,4,1,8,1),(1,384,4,3),(1,27,3,1,9,1),
            (1,60,3,1,4,1,5,1),(1,108,3,2,6,1),(1,1944,3,4),(1,20,2,1,10,1),(1,100,2,1,5,2),
            (1,48,2,1,4,1,6,1),(1,42,2,1,3,1,7,1),(1,144,2,1,3,2,4,1),(1,64,2,2,8,1),
            (1,256,2,2,4,2),(1,120,2,2,3,1,5,1),(1,288,2,3,6,1),(1,864,2,3,3,2),
            (1,1536,2,4,4,1),(1,46080,2,6),(1,11,1,1,11,1),(1,30,1,1,5,1,6,1),
            (1,28,1,1,4,1,7,1),(1,24,1,1,3,1,8,1),(1,96,1,1,3,1,4,2),(1,90,1,1,3,2,5,1),
            (1,18,1,1,2,1,9,1),(1,40,1,1,2,1,4,1,5,1),(1,36,1,1,2,1,3,1,6,1),
            (1,324,1,1,2,1,3,3),(1,56,1,1,2,2,7,1),(1,96,1,1,2,2,3,1,4,1),(1,240,1,1,2,3,5,1),
            (1,1152,1,1,2,4,3,1),(1,20,1,2,10,1),(1,100,1,2,5,2),(1,48,1,2,4,1,6,1),
            (1,42,1,2,3,1,7,1),(1,144,1,2,3,2,4,1),(1,32,1,2,2,1,8,1),(1,128,1,2,2,1,4,2),
            (1,60,1,2,2,1,3,1,5,1),(1,96,1,2,2,2,6,1),(1,288,1,2,2,2,3,2),(1,384,1,2,2,3,4,1),
            (1,7680,1,2,2,5),(1,54,1,3,9,1),(1,120,1,3,4,1,5,1),(1,108,1,3,3,1,6,1),
            (1,972,1,3,3,3),(1,84,1,3,2,1,7,1),(1,144,1,3,2,1,3,1,4,1),(1,240,1,3,2,2,5,1),
            (1,864,1,3,2,3,3,1),(1,192,1,4,8,1),(1,768,1,4,4,2),(1,360,1,4,3,1,5,1),
            (1,288,1,4,2,1,6,1),(1,864,1,4,2,1,3,2),(1,768,1,4,2,2,4,1),(1,9216,1,4,2,4),
            (1,840,1,5,7,1),(1,1440,1,5,3,1,4,1),(1,1200,1,5,2,1,5,1),(1,2880,1,5,2,2,3,1),
            (1,4320,1,6,6,1),(1,12960,1,6,3,2),(1,5760,1,6,2,1,4,1),(1,34560,1,6,2,3),
            (1,25200,1,7,5,1),(1,30240,1,7,2,1,3,1),(1,161280,1,8,4,1),(1,322560,1,8,2,2),
            (1,1088640,1,9,3,1),(1,7257600,1,10,2,1),(1,479001600,1,12)]
            }
    
    # set cycle index dimension based on w and h
    # and set up emtpy output with w*h empty tuples
    W = cycle_index[w]
    H = cycle_index[h]
    output = [[] for t in range(len(W)*len(H))]
    
    # set first counter to zero
    index = 0
    
    # run through each tuple in W against each tuple in H
    # each WH combination makes a new tuple in output
    for i in W:
        for j in H:
            output[index].append((long(i[0])*long(j[0]))) # maybe make long int's
            output[index].append((long(i[1])*long(j[1]))) # maybe make long int's
            for x in range(2,len(i),2):  # cycle through pairs of index monomial coefficients
                for y in range(2,len(j),2):  # same as for x above
                    output[index].append(long(lcm(i[x],j[y])))
                    output[index].append(long(i[x+1])*long(j[y+1])*\
                                         long(gcd(i[x],j[y])))
            index += 1
            
    # set result variable to a fraction
    result = Fraction(long(0),long(1))
    # add each term as fractions
    for i in output:
        result += Fraction(long(i[0]*long(pow(s,sum([long(i[j]) for j in range(3,len(i),2)])))),
                           long(i[1]))
        
    # for now return fraction, will need to convert to string after testing complete
    return result

# Test Function

In [12]:
w = 2
h = 3
s = 4

In [13]:
answer(w,h,s)

Fraction(430, 1)